In [35]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [36]:
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info = True , as_supervised=True)

In [37]:
mnist_dataset

{'train': <PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 'test': <PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}

In [21]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_path='C:\\Users\\Asus\\tensorflow_datasets\\mnist\\3.0.1',
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
)

In [22]:
mnist_test, mnist_train = mnist_dataset['test'], mnist_dataset['train']

In [23]:
mnist_test

<PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [24]:
mnist_train

<PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [25]:
num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

In [26]:
num_validation_samples

<tf.Tensor: shape=(), dtype=int64, numpy=6000>

In [27]:
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [28]:
num_test_samples

<tf.Tensor: shape=(), dtype=int64, numpy=10000>

In [29]:
print(num_test_samples)

tf.Tensor(10000, shape=(), dtype=int64)


In [30]:
def scale(image, label):
    image = tf.cast(image,tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)
buffer_size = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(buffer_size)
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

batch_size = 50
train_data = train_data.batch(batch_size)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)
validation_inputs, validation_targets = next(iter(validation_data))


In [31]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
                                tf.keras.layers.Flatten(input_shape=(28,28,1) ),
                                tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                                tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                                tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                                tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                                tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                                tf.keras.layers.Dense(output_size, activation='softmax')
    
    
])

In [32]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [33]:
NUM_EPOCHS = 5
model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), 
          verbose = 2)


Epoch 1/5
1080/1080 - 5s - loss: 0.2446 - accuracy: 0.9256 - val_loss: 0.1251 - val_accuracy: 0.9623 - 5s/epoch - 4ms/step
Epoch 2/5
1080/1080 - 3s - loss: 0.1141 - accuracy: 0.9665 - val_loss: 0.0897 - val_accuracy: 0.9708 - 3s/epoch - 3ms/step
Epoch 3/5
1080/1080 - 3s - loss: 0.0829 - accuracy: 0.9749 - val_loss: 0.0734 - val_accuracy: 0.9782 - 3s/epoch - 3ms/step
Epoch 4/5
1080/1080 - 3s - loss: 0.0674 - accuracy: 0.9798 - val_loss: 0.0739 - val_accuracy: 0.9778 - 3s/epoch - 3ms/step
Epoch 5/5
1080/1080 - 3s - loss: 0.0562 - accuracy: 0.9830 - val_loss: 0.0543 - val_accuracy: 0.9847 - 3s/epoch - 3ms/step


In [34]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 265ms/step - loss: 0.0853 - accuracy: 0.9749
